In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# pip install ipykernel plotly_express nbformat

#log_directory = Path('/logs/2024-10-19/atlas1/dcaade234ad54ba4931241ff6fe5fa96')
#log_directory = Path('/logs/2024-10-21/atlas1/c3c36fdf717b4ff88bca11eddd966c94')
log_directory = Path('/logs/2024-10-21/run_local/724fd6728d4149c09a54880480118937')


time_data_file = log_directory / 'msg_times.bin'

time_entry = np.dtype([('elapsed_ms', np.uint32), ('seq_num', np.uint32)])

time_data = np.fromfile(time_data_file, time_entry)
df = pd.DataFrame(time_data)
print(df)

In [ ]:
import plotly.express as px

df['dt'] = df['elapsed_ms'].diff()

fig = px.scatter(df, x='elapsed_ms', y='dt')
fig.show()

In [ ]:
from fusion_engine_client.parsers import fast_indexer
from fusion_engine_client.messages import PoseMessage, MessageHeader

input_data_file = log_directory / 'input.raw'
index = fast_indexer.fast_generate_index(str(input_data_file))

print(len(df['seq_num']), len(index.offset))

# The msg_times.bin stops before the end of the data sequence (it covers the messages that were actually checked).
index = index[:len(df['seq_num'])]

# Check sequence numbers
header = MessageHeader()
header_size = MessageHeader.calcsize()
with open(input_data_file, 'rb') as fd:
    for i in range(len(df['seq_num'])):
        offset = index.offset[i]
        fd.seek(offset)
        data = fd.read(header_size)
        header.unpack(data)
        assert header.sequence_number == df['seq_num'][i]


pose_indexes = index[PoseMessage.MESSAGE_TYPE].message_index
pose_timestamps = (df['elapsed_ms'].to_numpy())[pose_indexes]

fig = px.scatter(x=pose_timestamps[1:], y=np.diff(pose_timestamps))
fig.show()
